<a href="https://colab.research.google.com/github/peiva-git/deep_learning_project/blob/44-separare-i-notebook-in-file-separati/simple_ae_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports, definitions and setup

In [ ]:
!git clone https://github.com/peiva-git/deep_learning_project.git
%cd deep_learning_project

In [2]:
import dlproject as dlp
import matplotlib.pyplot as plt
import os.path
import numpy as np

# Model with CIFAR10 dataset

## Load the CIFAR10 dataset

In [ ]:
dataset_builder = dlp.data.CIFAR10DatasetBuilder()
dataset_builder.preprocess_dataset_simple_ae(0.1)
train_data, test_data = dataset_builder.train_x, dataset_builder.test_x
noisy_train_data, noisy_test_data = dataset_builder.noisy_train_data, dataset_builder.noisy_test_data

## Istantiate the model for CIFAR10 dataset

In [4]:
autoencoder_cifar = dlp.models.SimpleAutoencoder(input_shape=(32, 32, 3))
autoencoder_cifar.model.compile(optimizer='adam', loss='binary_crossentropy')
model = autoencoder_cifar.model

# Train the CIFAR10 model



In [5]:
from tensorflow.keras.saving import load_model

model_path = os.path.join(os.getcwd(), 'models', f'{model.name}_cifar_low_noise.keras')

if os.path.exists(model_path):
  model = load_model(model_path)
else:
  if not os.path.exists(os.path.join(os.getcwd(), 'models')):
    os.mkdir(os.path.join(os.getcwd(), 'models'))

  model.fit(
      x=noisy_train_data,
      y=train_data,
      epochs=100,
      batch_size=128,
      shuffle=True,
      validation_data=(noisy_test_data, test_data)
  )
  autoencoder_cifar.model.save(os.path.join(os.getcwd(), 'models', f'{model.name}_cifar.keras'))

Display some results.

In [ ]:
reconstructed_images = model.predict(noisy_test_data)
dlp.data.visualize.display_random_images(test_data, noisy_test_data, reconstructed_images, num_images=10)

Compute and display the PSNR and SSIM metrics.

In [ ]:
print(dlp.evaluation.compute_mean_psnr(test_data, reconstructed_images))
print(dlp.evaluation.compute_mean_ssim(test_data, reconstructed_images))